### Initialization & load initial events and items

In [1]:
# import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

In [3]:
items = pd.read_parquet('goodsread/items.parquet')
events = pd.read_parquet('goodsread/events.parquet')

### Урок 4/6 Холодный старт

Задание 2 из 6     
Завершите код для разбиения всех событий. В качестве точки разбиения используйте 2017-08-01, то есть отнесите в тестовую часть три последних месяца.

In [4]:
# зададим точку разбиения
train_test_global_time_split_date = pd.to_datetime('2017-08-01').date()

train_test_global_time_split_idx = (
    events["started_at"] < train_test_global_time_split_date
)

events_train = events[train_test_global_time_split_idx]
events_test = events[~train_test_global_time_split_idx]

# количество пользователей в train и test
users_train = events_train['user_id'].drop_duplicates()
users_test = events_test['user_id'].drop_duplicates()

# количество пользователей, которые есть и в train, и в test
common_users = set(users_train) & set(users_test)

print(len(users_train), len(users_test), len(common_users))

# Приборка
del train_test_global_time_split_idx, train_test_global_time_split_date, events

428220 123223 120858


Задание 3 из 6    
Идентифицируйте холодных пользователей и оцените их количество.

In [5]:
cold_users = set(users_test) - common_users

print(len(cold_users))

2365


Задание 4 из 6   
Завершите код, чтобы получить топ-100 наиболее популярных книг согласно условиям выше.

In [6]:
top_pop_start_date = pd.to_datetime('2015-01-01').date()

item_popularity = (
    events_train # type: ignore
    .query('started_at >= @top_pop_start_date')
    .groupby(['item_id'])
    .agg(users=('user_id', 'nunique'), avg_rating=('rating', 'mean'))
    .reset_index()
)
item_popularity['popularity_weighted'] = (
    item_popularity['users'] * item_popularity['avg_rating']
)

del top_pop_start_date
item_popularity.head(3)

,item_id,users,avg_rating,popularity_weighted
0,1,8728,4.796059,41860.0
1,2,9528,4.685873,44647.0
2,3,15139,4.706057,71245.0


In [7]:
# сортируем по убыванию взвешенной популярности
item_popularity = item_popularity.sort_values(
    'popularity_weighted',
    ascending=False
)
# выбираем первые 100 айтемов со средней оценкой avg_rating не меньше 4
top_k_pop_items = (
    item_popularity
    .query('avg_rating >= 4')
    .sort_values('popularity_weighted', ascending=False)
    .head(100)
)
top_k_pop_items.head(3)

,item_id,users,avg_rating,popularity_weighted
32387,18007564,20207,4.321275,87320.0
32623,18143977,19462,4.290669,83505.0
30695,16096824,16770,4.301014,72128.0


In [8]:
# добавляем информацию о книгах
if not set(top_k_pop_items.columns) & {
    "author", "title", "genre_and_votes", "publication_year"
}:
    top_k_pop_items = top_k_pop_items.merge(
        items.set_index("item_id")[
            ["author", "title", "genre_and_votes", "publication_year"]
        ],
        on="item_id",
        how='left'
    )
top_k_pop_items.head(3)

,item_id,users,avg_rating,popularity_weighted,author,title,genre_and_votes,publication_year
0,18007564,20207,4.321275,87320.0,Andy Weir,The Martian,"{'Science Fiction': 11966, 'Fiction': 8430}",2014
1,18143977,19462,4.290669,83505.0,Anthony Doerr,All the Light We Cannot See,"{'Historical-Historical Fiction': 13679, 'Fict...",2014
2,16096824,16770,4.301014,72128.0,Sarah J. Maas,A Court of Thorns and Roses (A Court of Thorns...,"{'Fantasy': 14326, 'Young Adult': 4662, 'Roman...",2015


Задание 5 из 6    
Завершите предлагаемый код, чтобы в cold_users_events_with_recs для каждого события получить столбец avg_rating. В нём при совпадении по item_id будет значение из одноимённого столбца из top_k_pop_items, иначе — пропуск.
В cold_users_events_with_recs после выполнения завершённого кода должно быть столько же строк, сколько было до его выполнения.

In [9]:
cold_users_events_with_recs = \
    events_test[events_test["user_id"].isin(cold_users)] \
    .merge(top_k_pop_items[['item_id', 'avg_rating']], on="item_id", how="left")

cold_user_items_no_avg_rating_idx = cold_users_events_with_recs["avg_rating"].isnull()
cold_user_recs = cold_users_events_with_recs[~cold_user_items_no_avg_rating_idx] \
    [["user_id", "item_id", "rating", "avg_rating"]] 

In [10]:
len(cold_user_recs) / len(cold_users_events_with_recs)

0.19768403639371382

Задание 6    
Посчитайте метрики rmse и mae для полученных рекомендаций.

In [11]:
# посчитаем метрики рекомендаций
from sklearn.metrics import mean_squared_error, mean_absolute_error

rmse = mean_squared_error(cold_user_recs["rating"], cold_user_recs["avg_rating"], squared=False)
mae = mean_absolute_error(cold_user_recs["rating"], cold_user_recs["avg_rating"])
print(round(rmse, 2), round(mae, 2)) 

0.78 0.62


In [12]:
# посчитаем покрытие холодных пользователей рекомендациями

cold_users_hit_ratio = cold_users_events_with_recs.groupby("user_id").agg(hits=("avg_rating", lambda x: (~x.isnull()).mean()))

print(f"Доля пользователей без релевантных рекомендаций: {(cold_users_hit_ratio == 0).mean().iat[0]:.2f}")
print(f"Среднее покрытие пользователей: {cold_users_hit_ratio[cold_users_hit_ratio != 0].mean().iat[0]:.2f}") 

Доля пользователей без релевантных рекомендаций: 0.59
Среднее покрытие пользователей: 0.44


In [13]:
# сохрянаяем важные переменные
# events_test.to_parquet('cache/events_test.parquet')
# events_train.to_parquet('cache/events_train.parquet')

In [14]:
# Приборка
for var_name in (    
    'cold_user_items_no_avg_rating_idx',
    'cold_user_recs',
    'cold_users',
    'cold_users_events_with_recs',
    'cold_users_hit_ratio',
    'common_users',
    'item_popularity',
    'rmse',
    'mae',
    'top_k_pop_items',
    'users_test',
    'users_train'
):
    if locals().get(var_name) is not None:
        del locals()[var_name]
del var_name